# __Classification__

Learning Outcomes:
- Apply SVC to data sets to predict binary outcome measures (groups)
- Explore different metrics to evaluate the model performance in classification settings and visualise variable importance
- Investigate the effect of using different types of kernels on the model performance
- Apply SVMs in a regression setting using SVR

In [ ]:
 # IMPORTS
    
from sklearn.svm import SVC, LinearSVC, NuSVC, SVR, NuSVR, LinearSVR
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.metrics import roc_curve, confusion_matrix, roc_auc_score, f1_score, accuracy_score, balanced_accuracy_score, classification_report, r2_score


In [ ]:
diabetes_metabol_plasma = pd.read_excel('../Data/diabetes_metabolomics_plasma.xlsx')
diabetes_metabol_saliva = pd.read_excel('../Data/diabetes_metabolomics_saliva.xlsx')

In [ ]:
display(diabetes_metabol_plasma)
display(diabetes_metabol_plasma.T2D.value_counts())


### Preprocessing 

In [ ]:
# Split the data into train and test sets
X_train_unscaled, X_test_unscaled, y_train, y_test = train_test_split(diabetes_metabol_plasma.iloc[:, 6:], diabetes_metabol_plasma.T2D, test_size=0.2, random_state=56)


If you `display(X_test_unscaled, y_test)` you will notice that the function retains the index values for each sample, so you can check that the splitting and the respective `y` targets (T2D status) have been split correctly. 

In [ ]:
# Scale the data with standard scaling (0 mean and unit variance)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_unscaled)
X_test = scaler.transform(X_test_unscaled)

## Support Vector Classifiers

In [ ]:
# Define the classifier with the standard parameters set by sklearn 
clf = SVC(kernel='rbf')

# Fit your first SVC model 
clf.fit(X_train, y_train)

# Predict the test set 
y_pred = clf.predict(X_test)


In [ ]:
# Now let's try with a linear kernel...
clf_linear = LinearSVC(C=1.4)

# Fit your first SVC model 
clf_linear.fit(X_train, y_train)

# Predict the test set 
y_pred_linear = clf_linear.predict(X_test)

### Accuracy metrics

We will explore a variety of metrics commonly used to evaluate classification models. The functions we will use are all found under `sklearn.metrics`. Please take some time to read the [documentation](https://scikit-learn.org/stable/modules/model_evaluation.html) and familiarise yourself with the differences between accuracy, AUC (Area Under the ROC Curve), f1 score, precision, recall etc. 

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
display(accuracy)

In [ ]:
ax = plt.subplot()
cm = confusion_matrix(y_test, y_pred)
display(cm)
sns.heatmap(cm, annot=True, ax=ax, cmap='Greens'); #annot=True to annotate cells
ax.set_xlabel('Predicted status')
ax.set_ylabel('True status')
ax.set_title('Confusion Matrix')

# in this case 0 is Healthy and 1 is T2D samples therefore we can name the labels 
ax.xaxis.set_ticklabels(['Healthy', 'T2D'])
ax.yaxis.set_ticklabels(['Healthy', 'T2D'])

In [ ]:
auc = roc_auc_score(y_test, clf.decision_function(X_test), average='macro')
display(auc)

auc = roc_auc_score(y_test, clf.decision_function(X_test), average='micro')
display(auc)

auc = roc_auc_score(y_test, clf.decision_function(X_test), average='weighted')
display(auc)

In [ ]:
roc = roc_curve(y_test, clf.decision_function(X_test))
display(roc)
fpr = roc[0]
tpr = roc[1]

In [ ]:
sns.lineplot(fpr,tpr,lw=2,label="ROC curve (area = %0.2f)" % auc, ci=None)
sns.lineplot([0, 1], [0, 1], lw=2, linestyle="--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC")
plt.legend(loc="lower right")


In [ ]:
f1 = f1_score(y_test, y_pred)
display(f1)

In [ ]:
# Try calculating the above metrics for the Linear kernel predictions...



### Parameter optimisation

You will notice that the `SVC()` function used above contains a number of parameters that need to be set by the user in order to optimise the model. The parameter `C` functions as a regularisation parameter and non-linear kernels (like the radial basis function (`rbf`) that we used above) require the `gamma` parameter which defines the kernel coefficient. For a more detailed explanation, you can study the kernel functions further on the [documentation](https://scikit-learn.org/stable/modules/svm.html#svm-kernels).

We will use the [`GridSearchCV()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) function to find the optimal parameters that maximise the accuracy of our model. 

In [ ]:
# define parameters range or set more than one sklearn-given options
param_grid = {'C': np.logspace(-2, 10, 13), 
              'gamma': ['scale', 'auto'], 
              'kernel': ['linear', 'poly', 'rbf', 'sigmoid']
             }

gridcv = GridSearchCV(SVC(), param_grid, refit=True, cv=5, verbose=1, n_jobs=-1)


# fit the model for grid search 
gridcv.fit(X_train, y_train) 
 
# display best parameters after tuning 
display(gridcv.best_params_) 
gridcv_pred = gridcv.predict(X_test) 
   
# classification report 
print(classification_report(y_test, gridcv_pred)) 

### Feature Selection

Ofter when we have a lot of features in our dataset we want to eliminate those that do not contribute much to the classification, or select the featues that drive the classification.One of the most used feature selection techniques is __Recursive Feature Elimination (RFE)__. Given an external estimator that assigns weights to features (e.g., the coefficients of a linear model), the goal of recursive feature elimination (RFE) is to select features by recursively considering smaller and smaller sets of features. First, the estimator is trained on the initial set of features and the importance of each feature is obtained either through any specific attribute or callable. Then, the least important features are pruned from current set of features. That procedure is recursively repeated on the pruned set until the desired number of features to select is eventually reached. To find out more about RFE or other feature selection methods, read thourgh the [documentation](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_selection). Here we will use the cross-validation RFE function [`RFECV()`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFECV.html#sklearn.feature_selection.RFECV).   

We will use the [`NuSVC()`](https://scikit-learn.org/stable/modules/generated/sklearn.svm.NuSVC.html) function which is similar to the previously used SVC but uses the `nu` parameter to control the number of support vectors. 

In [ ]:
# define the classifier of choice and its parameters
clf = NuSVC(
    nu=0.5, 
    kernel='linear', 
    gamma='scale'
)

# create the rfe object
rfecv = RFECV(
    estimator=clf,
    step=1,
    cv=5,
    scoring="accuracy",
    min_features_to_select=1,
)

%time rfecv.fit(X_train, y_train)

display('optimal n of features: %d' % rfecv.n_features_)


In [ ]:
# plot the change in accuracy when looking at the step-wise different number of features
plt.xlabel("number of features selected")
plt.ylabel("accuracy")
for i in range(0,5):
    sns.lineplot(
        range(1, len(rfecv.grid_scores_) + 1),
        rfecv.grid_scores_[:, i],
        label=f'CV fold {i+1}'
    )

### __Your turn...__

Try defining a new classifier with the [`LinearSVC()`](https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html) function and perform a grid search to decide the best `C` values for the model. 

Then apply RFE cross-validation and use the optimal number of features and the selected features to run your model. _Hint_: to get the selected features use the `rfecv.support_` attribute which gives you a mask [True (selected), False (not selected)] array of all the features.  


In [ ]:
# define the classifier 

# set the parameter grid 

# create the rfe object



#### Regression

Load an appropriate dataset with continuous target variables and repeat the above parts of the tutorial for Support Vector Regression models.

The [`SVR()`](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html) function requires the same parameters that you encountered before (C, kernel, gamma (for non-linear kernels) etc) with an additional parameter `epsilon` being added. It specifies the epsilon-tube within which no penalty is associated in the training loss function with points predicted within a distance epsilon from the actual value. Similar to the classification objects we used above, sklearn defines [`NuSVR()`](https://scikit-learn.org/stable/modules/generated/sklearn.svm.NuSVR.html#sklearn.svm.NuSVR) and [`LinearSVR()`](https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVR.html#sklearn.svm.LinearSVR) objects for regression tasks as well. 

In [ ]:
# split the data into train and test sets; the BMI values which will be our target (y) for this regression example 
X_train_reg_unscaled, X_test_reg_unscaled, y_train_reg, y_test_reg = train_test_split(diabetes_metabol_plasma.iloc[:,6:], diabetes_metabol_plasma.BMI, test_size=0.2, random_state=56)

# scaling 
scaler = StandardScaler()
X_train_reg = scaler.fit_transform(X_train_reg_unscaled)
X_test_reg = scaler.transform(X_test_reg_unscaled)


In [ ]:
# define the regressor with the default sklearn parameters 
regr = SVR() # here the epsilon value is the default 0.1

regr.fit(X_train_reg, y_train_reg)
y_pred_reg = regr.predict(X_test_reg)

r2 = r2_score(y_test_reg, y_pred_reg) # another metric used in regression is Mean Squared Error (MSE)

# try to plot these results 


_Hint:_ sklearn has a few good examples for plotting results -- a few to look at is this [comparison of kernel ridge regression and SVR](https://scikit-learn.org/stable/auto_examples/miscellaneous/plot_kernel_ridge_regression.html#sphx-glr-auto-examples-miscellaneous-plot-kernel-ridge-regression-py) and the examples on [model complexity influence](https://scikit-learn.org/stable/auto_examples/applications/plot_model_complexity_influence.html#sphx-glr-auto-examples-applications-plot-model-complexity-influence-py)

In addition make sure to investigate the different metrics used to evaluate regression models, some examples include the r2 score, mean squared error, mean absolute error, and others.  

